# Introduktion til Sandsynlighed og Statistik
I denne notesbog vil vi lege med nogle af de begreber, vi tidligere har diskuteret. Mange begreber inden for sandsynlighed og statistik er godt repræsenteret i store biblioteker til databehandling i Python, såsom `numpy` og `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Tilfældige variable og fordelinger
Lad os starte med at trække et stikprøve af 30 værdier fra en ensartet fordeling fra 0 til 9. Vi vil også beregne middelværdi og varians.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

For visuelt at estimere, hvor mange forskellige værdier der er i prøven, kan vi tegne **histogrammet**:


In [ ]:
plt.hist(sample)
plt.show()

## Analyse af rigtige data

Gennemsnit og varians er meget vigtige, når man analyserer virkelige data. Lad os indlæse dataene om baseballspillere fra [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Vi bruger en pakke kaldet [**Pandas**](https://pandas.pydata.org/) her til dataanalyse. Vi vil tale mere om Pandas og arbejde med data i Python senere i dette kursus.

Lad os beregne gennemsnitsværdier for alder, højde og vægt:


In [ ]:
df[['Age','Height','Weight']].mean()

Lad os nu fokusere på højde og beregne standardafvigelse og varians:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Ud over gennemsnittet giver det mening at se på medianværdien og kvartilerne. De kan visualiseres ved hjælp af et **box plot**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Vi kan også lave boksplot af delmængder af vores datasæt, for eksempel grupperet efter spillerrolle.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Bemærk**: Dette diagram antyder, at højderne på første basmænd i gennemsnit er højere end højderne på anden basmænd. Senere vil vi lære, hvordan vi mere formelt kan teste denne hypotese, og hvordan vi kan demonstrere, at vores data er statistisk signifikante til at vise dette.  

Alder, højde og vægt er alle kontinuerlige stokastiske variable. Hvad tror du deres fordeling er? En god måde at finde ud af det på er at tegne et histogram over værdierne: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Normalfordeling

Lad os lave en kunstig prøve af vægte, der følger en normalfordeling med samme gennemsnit og varians som vores rigtige data:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Da de fleste værdier i det virkelige liv er normalfordelte, bør vi ikke bruge en jævnt fordelt tilfældig talgenerator til at generere prøve-data. Her er, hvad der sker, hvis vi prøver at generere vægte med en jævn fordeling (genereret af `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Konfidensintervaller

Lad os nu beregne konfidensintervaller for vægtene og højderne på baseballspillere. Vi vil bruge koden [fra denne stackoverflow-diskussion](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Hypotesetestning

Lad os udforske forskellige roller i vores baseballspillere-datasæt:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Lad os teste hypotesen om, at førstekædemænd er højere end andenkædemænd. Den nemmeste måde at gøre dette på er at teste konfidensintervallerne:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Vi kan se, at intervallerne ikke overlapper.

En statistisk mere korrekt måde at bevise hypotesen på er at bruge en **Student t-test**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

De to værdier, der returneres af funktionen `ttest_ind`, er:
* p-værdien kan betragtes som sandsynligheden for, at to fordelinger har samme middelværdi. I vores tilfælde er den meget lav, hvilket betyder, at der er stærke beviser for, at førstekommere er højere.
* t-værdien er den mellemste værdi af den normaliserede middelværdi-forskel, der bruges i t-testen, og den sammenlignes med en tærskelværdi for en given konfidensværdi.


## Simulering af en normalfordeling med centralgrænseværdisætningen

Den pseudo-tilfældige generator i Python er designet til at give os en ensartet fordeling. Hvis vi ønsker at skabe en generator for normalfordeling, kan vi bruge centralgrænseværdisætningen. For at få en normalfordelt værdi vil vi blot beregne gennemsnittet af et prøveudtagning genereret med en uniform fordeling.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Korrelation og Evil Baseball Corp

Korrelation gør det muligt for os at finde relationer mellem datasekvenser. I vores legeteksempel, lad os forestille os, at der er en ond baseball-virksomhed, som betaler sine spillere efter deres højde - jo højere spilleren er, desto flere penge får han/hun. Antag, at der er en grundløn på $1000, og en ekstra bonus fra $0 til $100, afhængigt af højden. Vi vil tage de rigtige spillere fra MLB og beregne deres fiktive lønninger:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Lad os nu beregne kovarians og korrelation af disse sekvenser. `np.cov` vil give os en såkaldt **kovariansmatrix**, som er en udvidelse af kovarians til flere variable. Elementet $M_{ij}$ i kovariansmatrixen $M$ er en korrelation mellem inputvariablerne $X_i$ og $X_j$, og de diagonale værdier $M_{ii}$ er variansen af $X_{i}$. Tilsvarende vil `np.corrcoef` give os **korrelationsmatrixen**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

En korrelation lig med 1 betyder, at der er en stærk **lineær sammenhæng** mellem to variable. Vi kan visuelt se den lineære sammenhæng ved at plotte den ene værdi mod den anden:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Lad os se, hvad der sker, hvis relationen ikke er lineær. Antag, at vores virksomhed besluttede at skjule den åbenlyse lineære afhængighed mellem højder og lønninger og indførte en vis ikke-linearitet i formlen, såsom `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

I dette tilfælde er korrelationen en smule mindre, men den er stadig ret høj. Nu, for at gøre relationen endnu mindre åbenlys, kunne vi tilføje noget ekstra tilfældighed ved at lægge en tilfældig variabel til lønnen. Lad os se, hvad der sker:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Kan du gætte, hvorfor prikkerne danner lodrette linjer som dette?

Vi har observeret korrelationen mellem et kunstigt konstrueret koncept som løn og den observerede variabel *højde*. Lad os også se, om de to observerede variable, såsom højde og vægt, også korrelerer:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Desværre fik vi ikke nogen resultater - kun nogle mærkelige `nan` værdier. Dette skyldes, at nogle af værdierne i vores serie er udefinerede, repræsenteret som `nan`, hvilket gør resultatet af operationen udefineret også. Ved at se på matricen kan vi se, at `Weight` er den problematiske kolonne, fordi selvkorrelationen mellem `Height` værdier er blevet beregnet.

> Dette eksempel viser vigtigheden af **dataklargøring** og **rengøring**. Uden ordentlige data kan vi ikke beregne noget.

Lad os bruge `fillna` metoden til at udfylde de manglende værdier og beregne korrelationen: 


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Der er faktisk en korrelation, men ikke en så stærk som i vores kunstige eksempel. Hvis vi ser på scatterplottet af den ene værdi mod den anden, ville sammenhængen være meget mindre åbenlys:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Konklusion

I denne notesbog har vi lært, hvordan man udfører grundlæggende operationer på data for at beregne statistiske funktioner. Vi ved nu, hvordan man bruger et solidt apparat af matematik og statistik for at bevise nogle hypoteser, og hvordan man beregner konfidensintervaller for vilkårlige variable givet en datasample.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Ansvarsfraskrivelse**:
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os intet ansvar for misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
